# Chapter 3: Dictionaries and Sets

- the dict type is not only widely used in our programs but also a fundametal part of Python implementation
- dictionaries are deplyoed in: module namespaces, class and instance attributes and function keyword arguments, ...
- due to their crucial role, Python dicts are highly optimized
- **Hash tables** are the engines behind Python's high performance dicts
- sets are also implemented with Hash tables

# Dictionary | Sets traits/features/consequences

- dict keys must be hashable objects | set elements must be hashable objects
- both have significant memory overhead (not space efficient) due to internal hash table being sparse
- key search is very fast | set membership testing is very efficient
- key ordering depends on insertion order | element ordering depends on insertion order
- adding items to a dict may change the order of existing items | adding elements to a set may change ordering as well

### What is hashable object?

- atomic immutable types (str, bytes, numeric types) are all hashable.
- frozen set is always hashable, because its elements must be hashable by definition
- tuple is hashable only if all its items are hashable
- user-defined types are hashable by default as their hash value is their id()

In [1]:
t1 = (1, 2, (3, 4))
hash(t1)

3794340727080330424

In [2]:
t2 = (1, 2, [3, 4]) # list not hashable
hash(t2)

TypeError: unhashable type: 'list'

In [3]:
t3 = (1, 2, frozenset([3, 4]))
hash(t3)

131250961768736263

- given these ground rules we can build dictionaries in several ways

In [4]:
a = dict(one=1, two=2)
b = {"one": 1, "two": 2}
c = dict(zip(["one", "two"], [1, 2]))
d = dict([("two", 2), ("one", 1)])
e = dict({"two": 2, "one": 1})
a == b == c == d == e

True

### Dict comprehension

In [5]:
dial_codes = [
    (44, "UK"),
    (33, "France"),
    (31, "Netherlands")
]

codes = {country: code for code, country in dial_codes}
codes

{'UK': 44, 'France': 33, 'Netherlands': 31}

In [6]:
{country.upper(): code for code, country in dial_codes} # country to upper case

{'UK': 44, 'FRANCE': 33, 'NETHERLANDS': 31}

In [7]:
{country.upper(): code for code, country in dial_codes if code > 31} # condition within comprehension expression

{'UK': 44, 'FRANCE': 33}

## Handling missing keys in dict

### get() method

In [8]:
car = {
  "model": "Mustang",
  "year": 1964
}
print(car)
print(car.get("colours", ["black", "blue"]))
print(car)

# just get the defualt value when key missing, no changes to the actual "car" dictionary

{'model': 'Mustang', 'year': 1964}
['black', 'blue']
{'model': 'Mustang', 'year': 1964}


### setdefault() method

In [9]:
car = {
  "model": "Mustang",
  "year": 1964
}
print(car)
car.setdefault("colours", ["black"]).append("blue")
print(car)

# actual "car" dictionary changed with the key:value being added

{'model': 'Mustang', 'year': 1964}
{'model': 'Mustang', 'year': 1964, 'colours': ['black', 'blue']}


- the result of the above line above is the same as running ...

In [10]:
car2 = {
  "model": "Mustang",
  "year": 1964
}
print(car2)

if "colours" not in car2:
    car2["colours"] = ["black"]
    print(car2)
car2["colours"].append("blue")
print(car2)

# actual "car" dictionary changed with the key:value being added, but more expensive than previous approach

{'model': 'Mustang', 'year': 1964}
{'model': 'Mustang', 'year': 1964, 'colours': ['black']}
{'model': 'Mustang', 'year': 1964, 'colours': ['black', 'blue']}


- ... except this code performs at least 2 searches for a key - three if it's not found
- while setdefault does it all in a **single lookup**

### defaultdict

- configured to create items on demand whenever a missing key is searched
- need to provide a callable or None to defaultdict

In [11]:
from collections import defaultdict
example = defaultdict(list)
print(example)
example["non_existing_key"] # default value is list
print(example)
example["another_key"].append("blue")
print(example)

defaultdict(<class 'list'>, {})
defaultdict(<class 'list'>, {'non_existing_key': []})
defaultdict(<class 'list'>, {'non_existing_key': [], 'another_key': ['blue']})


## Variations of dict

- besides defaultdict as we it is covered above

### collections.OrderedDict

- maintains keys in insertion order
- allowing iteration over items in predictable order
- popitem() pops the first item by default, but if specified popitem(last=True) -> pops last item added

### collections.ChainMap

- holds a list of mappings which can be searched as one
- lookup is performed on each mapping in order, and succeeds is the key is found in any of them
- used in interpreters for languages with nested scopes

### collections.Counter

- holds an integer count for each key

In [12]:
from collections import Counter
counter = Counter("racecar")
counter

Counter({'r': 2, 'a': 2, 'c': 2, 'e': 1})

### collections.UserDict

- designed to be subclassed and implemented by the user

# Set and frozenset

- collection of unique objects
- set elements must be hashable. The set type is not hashable, but frozenset is. Hence we can have frozenset elements in a set
- set operations given two sets a and b:  
    a | b -> union  
    a & b -> intersection  
    a - b -> difference  
- using these oprations reduce line count, execution of the program and makes the code easier to read

In [13]:
# example set
s = set([1,1, 2,2, 3,3])
print(s)

s2 = {1,1,2,2,3,3}
print(s2)

s3 = {1,1,2,2,3,3, frozenset([1,1, 2,2, 3,3])}
print(s3)

{1, 2, 3}
{1, 2, 3}
{frozenset({1, 2, 3}), 1, 2, 3}


- for example let's say we have a large set of e-mails (haystack) and a smaller set of addresses (needles) and we need to count how many of the needles occur in the haystack

- without intersection the code would look something like:

### Set comprehenstion

In [14]:
from unicodedata import name

# obtain character names using name function
# build a set of characters with codes from 100 to 255 that have the word 'SIGN' in their names
{chr(i) for i in range(100, 255) if "SIGN" in name(chr(i), '')}

{'¢', '£', '¤', '¥', '§', '©', '¬', '®', '°', '±', 'µ', '¶', '×', '÷'}

# The Hash Table algorithm

- hash table is a sparse array, i.e. an array which always has empty cells
- cells of the hash table are called **buckets**
- in a dict hash table, there is a bucket for each item, and it contains two fields:  
    a reference to the key  
    a reference to the value of the item
- because all buckets have the same size, access to an individual bucket is done by offset

To fetch value at my_dict[search_key], Python calls hash(search_key) to obtain the hash value of search_key and uses the least significant bits of that number as an offset to look up a bucket in the hash table (the number of bits used depends on the current size of the table).

If the found bucket is empty, KeyError is raised. Otherwise the found bucket has an item - a found_key:found_value pair - and then Python checks whether search_key == found_key. If they match, found_value is returned.

However, if search_key and found_key do not match, this is a hash collision. This happens because a hash function maps arbitrary objects to a small number of bits, and - in addition - the hash table is indexed with subset of those bits. To resolve the collision, the algorithm then takes different bits in the hash, arranges them in particular way and uses the result as an offset to look up a different bucket. If that is empty, KeyError is raised; if not, either the keys match and then the item value is returned, or the collision resolution process is repeated.

<img src="dict_hash.png">

# Bonus: inspecting set and dict literals

- set syntax: {1}, {1, 2}, etc looks exactly like math notation
- with one important exception: there is no literal for an empty set, we always need to specify set()

In [15]:
set1, set2 = {1}, {1,2}
print(set1, set2)
set2.pop()
print(set1, set2)

not_empty_set = {}
print(not_empty_set, type(not_empty_set))

empty_set = set()
print(empty_set, type(empty_set))

{1} {1, 2}
{1} {2}
{} <class 'dict'>
set() <class 'set'>


- using the literal is both faster and more readable than calling the constructor
- calling the constructor is slower because Python has to:  
    1) look up the set name to fetch the constructor  
    2) then build a list  
    3) and finally pass it to the constructor
    
- in contrast, to process the literal Python runs a specialized BUILD_SET bytecode

In [16]:
from dis import dis

dis('{1}') # literal

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [17]:
dis('set([1])') # constructor

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


### Similar applies to dict literals

In [18]:
dis('{"one": 1, "two": 2}')

  1           0 LOAD_CONST               0 (1)
              2 LOAD_CONST               1 (2)
              4 LOAD_CONST               2 (('one', 'two'))
              6 BUILD_CONST_KEY_MAP      2
              8 RETURN_VALUE


In [19]:
dis('dict([("two", 2), ("one", 1)])')

  1           0 LOAD_NAME                0 (dict)
              2 LOAD_CONST               0 (('two', 2))
              4 LOAD_CONST               1 (('one', 1))
              6 BUILD_LIST               2
              8 CALL_FUNCTION            1
             10 RETURN_VALUE
